In [1]:
# import libraries
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config_db import password_db
from flask import Flask, jsonify
import pandas as pd
import json

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [2]:
# call the database

connection_string = 'postgres:'+password_db+'@localhost:5432/travel_data'

engine=create_engine(f"postgresql://{connection_string}")

In [3]:
# reflect existing database
Base = automap_base()

In [4]:
# reflect the tables
Base.prepare(engine, reflect = True)

In [5]:
# save references to table
travel_details = Base.classes.travel_data_details()


In [6]:
travel_details

In [7]:
# verify table content
Travel_Details = Base.classes.travel_data_details
Travel_Details.__dict__

mappingproxy({'__table__': Table('travel_data_details', MetaData(bind=None), Column('index', INTEGER(), table=<travel_data_details>, primary_key=True, nullable=False), Column('date', VARCHAR(length=12), table=<travel_data_details>), Column('state_fips', INTEGER(), table=<travel_data_details>), Column('state_code', VARCHAR(length=2), table=<travel_data_details>), Column('pop_stay_at_home', INTEGER(), table=<travel_data_details>), Column('pop_not_stay_at_home', INTEGER(), table=<travel_data_details>), Column('trips', INTEGER(), table=<travel_data_details>), Column('trips_1', INTEGER(), table=<travel_data_details>), Column('trips_1_3', INTEGER(), table=<travel_data_details>), Column('trips_3_5', INTEGER(), table=<travel_data_details>), Column('trips_5_10', INTEGER(), table=<travel_data_details>), Column('trips_10_25', INTEGER(), table=<travel_data_details>), Column('trips_25_50', INTEGER(), table=<travel_data_details>), Column('trips_50_100', INTEGER(), table=<travel_data_details>), Colum

In [8]:
session = Session(engine)

In [9]:
# connect to sql
connection = engine.connect()

In [10]:
# query data needed for graphs
travel_trips_df = pd.read_sql("""select year, month, month_abbr, month_year, state, 
round(avg(pop_stay_at_home),0) as no_travel_pop, round(avg(pop_not_stay_at_home),0) as travel_pop, 
round(avg(trips),0) as total_trips, round(avg(trips_1),0) as trips_1, round(avg(trips_1_3),0) as trips_1_3,
round(avg(trips_3_5),0) as trips_3_5, round(avg(trips_5_10),0) as trips_5_10, round(avg(trips_10_25),0) as trips_10_25,
round(avg(trips_25_50),0) as trips_25_50, round(avg(trips_50_100),0) as trips_50_100,
round(avg(trips_100_250),0) as trips_100_250, round(avg(trips_250_500),0) as trips_250_500,
round(avg(trips_500),0) as trips_500 from travel_data_details 
GROUP BY state, month_year, year, month, month_abbr
order by state, year, month, month_year, month_abbr """, connection)
travel_trips_df


,year,month,month_abbr,month_year,state,no_travel_pop,travel_pop,total_trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
0,2019,1,Jan,Jan-2019,Alabama,817303.0,4055631.0,15694066.0,3457789.0,3928199.0,2102792.0,2550304.0,2381688.0,803007.0,297972.0,133914.0,29997.0,8404.0
1,2019,10,Oct,Oct-2019,Alabama,873602.0,3999332.0,17122849.0,3509941.0,4069888.0,2273704.0,2922332.0,3032815.0,969267.0,256227.0,67312.0,15779.0,5584.0
2,2019,11,Nov,Nov-2019,Alabama,863024.0,4009910.0,16051347.0,3222625.0,3741107.0,2160788.0,2737225.0,2880013.0,957496.0,266789.0,62945.0,16703.0,5656.0
3,2019,12,Dec,Dec-2019,Alabama,885949.0,3986985.0,16224584.0,3112266.0,3782532.0,2224508.0,2823810.0,2944228.0,972062.0,271608.0,68939.0,18240.0,6391.0
4,2019,2,Feb,Feb-2019,Alabama,786528.0,4086406.0,15782998.0,3406372.0,3917605.0,2100610.0,2574130.0,2433995.0,838147.0,321631.0,147557.0,31687.0,11265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,2020,5,May,May-2020,Wyoming,138418.0,439319.0,2161415.0,629094.0,590282.0,274332.0,214916.0,185114.0,145797.0,79799.0,36454.0,4334.0,1294.0
1046,2020,6,Jun,Jun-2020,Wyoming,130489.0,447248.0,2111551.0,600523.0,552089.0,259166.0,208627.0,190782.0,155855.0,90332.0,46482.0,5833.0,1863.0
1047,2020,7,Jul,Jul-2020,Wyoming,141108.0,436629.0,1887541.0,509499.0,491375.0,228916.0,190580.0,182780.0,146981.0,84735.0,44631.0,5770.0,2275.0
1048,2020,8,Aug,Aug-2020,Wyoming,139191.0,438546.0,1892930.0,490849.0,500760.0,232761.0,192435.0,182371.0,141622.0,90769.0,50868.0,6887.0,3608.0


In [11]:
# format measures to integer

travel_trips_df['no_travel_pop'] = travel_trips_df['no_travel_pop'].astype('int')
travel_trips_df['travel_pop'] = travel_trips_df['travel_pop'].astype('int')
travel_trips_df['total_trips'] = travel_trips_df['total_trips'].astype('int')
travel_trips_df['trips_1'] = travel_trips_df['trips_1'].astype('int')
travel_trips_df['trips_1_3'] = travel_trips_df['trips_1_3'].astype('int')
travel_trips_df['trips_3_5'] = travel_trips_df['trips_3_5'].astype('int')
travel_trips_df['trips_5_10'] = travel_trips_df['trips_5_10'].astype('int')
travel_trips_df['trips_10_25'] = travel_trips_df['trips_10_25'].astype('int')
travel_trips_df['trips_25_50'] = travel_trips_df['trips_25_50'].astype('int')
travel_trips_df['trips_50_100'] = travel_trips_df['trips_50_100'].astype('int')
travel_trips_df['trips_100_250'] = travel_trips_df['trips_100_250'].astype('int')
travel_trips_df['trips_250_500'] = travel_trips_df['trips_250_500'].astype('int')
travel_trips_df['trips_500'] = travel_trips_df['trips_500'].astype('int')

travel_trips_df

,year,month,month_abbr,month_year,state,no_travel_pop,travel_pop,total_trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
0,2019,1,Jan,Jan-2019,Alabama,817303,4055631,15694066,3457789,3928199,2102792,2550304,2381688,803007,297972,133914,29997,8404
1,2019,10,Oct,Oct-2019,Alabama,873602,3999332,17122849,3509941,4069888,2273704,2922332,3032815,969267,256227,67312,15779,5584
2,2019,11,Nov,Nov-2019,Alabama,863024,4009910,16051347,3222625,3741107,2160788,2737225,2880013,957496,266789,62945,16703,5656
3,2019,12,Dec,Dec-2019,Alabama,885949,3986985,16224584,3112266,3782532,2224508,2823810,2944228,972062,271608,68939,18240,6391
4,2019,2,Feb,Feb-2019,Alabama,786528,4086406,15782998,3406372,3917605,2100610,2574130,2433995,838147,321631,147557,31687,11265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,2020,5,May,May-2020,Wyoming,138418,439319,2161415,629094,590282,274332,214916,185114,145797,79799,36454,4334,1294
1046,2020,6,Jun,Jun-2020,Wyoming,130489,447248,2111551,600523,552089,259166,208627,190782,155855,90332,46482,5833,1863
1047,2020,7,Jul,Jul-2020,Wyoming,141108,436629,1887541,509499,491375,228916,190580,182780,146981,84735,44631,5770,2275
1048,2020,8,Aug,Aug-2020,Wyoming,139191,438546,1892930,490849,500760,232761,192435,182371,141622,90769,50868,6887,3608


In [12]:
# convert to json format

travel_detail_json = json.dumps(travel_trips_df.to_dict('records'))
travel_detail_json

'[{"year": "2019", "month": "1", "month_abbr": "Jan", "month_year": "Jan-2019", "state": "Alabama", "no_travel_pop": 817303, "travel_pop": 4055631, "total_trips": 15694066, "trips_1": 3457789, "trips_1_3": 3928199, "trips_3_5": 2102792, "trips_5_10": 2550304, "trips_10_25": 2381688, "trips_25_50": 803007, "trips_50_100": 297972, "trips_100_250": 133914, "trips_250_500": 29997, "trips_500": 8404}, {"year": "2019", "month": "10", "month_abbr": "Oct", "month_year": "Oct-2019", "state": "Alabama", "no_travel_pop": 873602, "travel_pop": 3999332, "total_trips": 17122849, "trips_1": 3509941, "trips_1_3": 4069888, "trips_3_5": 2273704, "trips_5_10": 2922332, "trips_10_25": 3032815, "trips_25_50": 969267, "trips_50_100": 256227, "trips_100_250": 67312, "trips_250_500": 15779, "trips_500": 5584}, {"year": "2019", "month": "11", "month_abbr": "Nov", "month_year": "Nov-2019", "state": "Alabama", "no_travel_pop": 863024, "travel_pop": 4009910, "total_trips": 16051347, "trips_1": 3222625, "trips_1_3

In [13]:
# # for row in session.query(Travel_Details.month, Travel_Details.state,Travel_Details.pop_stay_at_home):
# #     print(row)

# no_travel = session.query(Travel_Details.year, Travel_Details.month, Travel_Details.state,func.avg(Travel_Details.pop_stay_at_home)).\
# group_by(Travel_Details.year, Travel_Details.month,Travel_Details.state).\
# order_by(Travel_Details.year, Travel_Details.state,Travel_Details.month).all()

# #     no_travel = session.query(Travel_Details.month, Travel_Details.state,func.avg(Travel_Details.pop_stay_at_home)).\
# #     group_by(Travel_Details.month,Travel_Details.state).\
# #     order_by(Travel_Details.state,Travel_Details.month).all()

# no_travel
    


In [14]:
# no_travel = [(int(no_travel_tuple[0]), int(no_travel_tuple[1]), no_travel_tuple[2], round(float(no_travel_tuple[3])))\
#              for no_travel_tuple in no_travel]
# no_travel

In [15]:
# def row2dict(row):
#     d = {}
#     for column in row.__table__.columns:
#         d[column.name] = str(getattr(row, column.name))

#     return d

In [16]:
# session=Session(engine)

In [17]:
# geomap_travel = session.query(Travel_Details.year, Travel_Details.month, Travel_Details.state, Travel_Details.latitude,\
#                               Travel_Details.longitude, func.avg(Travel_Details.pop_not_stay_at_home)).\
#                               group_by(Travel_Details.year, Travel_Details.month,Travel_Details.state, \
#                                        Travel_Details.latitude, Travel_Details.longitude).\
#                               order_by(Travel_Details.year, Travel_Details.state,Travel_Details.month, \
#                                        Travel_Details.latitude, Travel_Details.longitude).all()
# geomap_travel

In [18]:
# geomap_travel = [(int(gp_travel_tuple[0]), int(gp_travel_tuple[1]), gp_travel_tuple[2], gp_travel_tuple[3], \
#                   gp_travel_tuple[4], round(float(gp_travel_tuple[5]))) for gp_travel_tuple in geomap_travel]
# geomap_travel

In [19]:
# read data for summary graph

travel_summary_df = pd.read_sql("""select year, month, month_abbr, month_year, state, latitude, longitude,
round(avg(pop_stay_at_home),0) as no_travel_pop, round(avg(pop_not_stay_at_home),0) as travel_pop from travel_data_details 
GROUP BY state, month_year, year, month, month_abbr, latitude, longitude
order by state, year, month, month_year, month_abbr """, connection)

travel_summary_df

,year,month,month_abbr,month_year,state,latitude,longitude,no_travel_pop,travel_pop
0,2019,1,Jan,Jan-2019,Alabama,32.806671,-86.79113,817303.0,4055631.0
1,2019,10,Oct,Oct-2019,Alabama,32.806671,-86.79113,873602.0,3999332.0
2,2019,11,Nov,Nov-2019,Alabama,32.806671,-86.79113,863024.0,4009910.0
3,2019,12,Dec,Dec-2019,Alabama,32.806671,-86.79113,885949.0,3986985.0
4,2019,2,Feb,Feb-2019,Alabama,32.806671,-86.79113,786528.0,4086406.0
...,...,...,...,...,...,...,...,...,...
1045,2020,5,May,May-2020,Wyoming,42.755966,-107.30249,138418.0,439319.0
1046,2020,6,Jun,Jun-2020,Wyoming,42.755966,-107.30249,130489.0,447248.0
1047,2020,7,Jul,Jul-2020,Wyoming,42.755966,-107.30249,141108.0,436629.0
1048,2020,8,Aug,Aug-2020,Wyoming,42.755966,-107.30249,139191.0,438546.0


In [20]:
# convert datatype to integer

travel_summary_df['no_travel_pop'] = travel_summary_df['no_travel_pop'].astype('int')
travel_summary_df['travel_pop'] = travel_summary_df['travel_pop'].astype('int')
travel_summary_df

,year,month,month_abbr,month_year,state,latitude,longitude,no_travel_pop,travel_pop
0,2019,1,Jan,Jan-2019,Alabama,32.806671,-86.79113,817303,4055631
1,2019,10,Oct,Oct-2019,Alabama,32.806671,-86.79113,873602,3999332
2,2019,11,Nov,Nov-2019,Alabama,32.806671,-86.79113,863024,4009910
3,2019,12,Dec,Dec-2019,Alabama,32.806671,-86.79113,885949,3986985
4,2019,2,Feb,Feb-2019,Alabama,32.806671,-86.79113,786528,4086406
...,...,...,...,...,...,...,...,...,...
1045,2020,5,May,May-2020,Wyoming,42.755966,-107.30249,138418,439319
1046,2020,6,Jun,Jun-2020,Wyoming,42.755966,-107.30249,130489,447248
1047,2020,7,Jul,Jul-2020,Wyoming,42.755966,-107.30249,141108,436629
1048,2020,8,Aug,Aug-2020,Wyoming,42.755966,-107.30249,139191,438546


In [21]:
# convert dataframe to json

travel_summary_json = json.dumps(travel_summary_df.to_dict('records'))
travel_summary_json

'[{"year": "2019", "month": "1", "month_abbr": "Jan", "month_year": "Jan-2019", "state": "Alabama", "latitude": 32.806671, "longitude": -86.79113, "no_travel_pop": 817303, "travel_pop": 4055631}, {"year": "2019", "month": "10", "month_abbr": "Oct", "month_year": "Oct-2019", "state": "Alabama", "latitude": 32.806671, "longitude": -86.79113, "no_travel_pop": 873602, "travel_pop": 3999332}, {"year": "2019", "month": "11", "month_abbr": "Nov", "month_year": "Nov-2019", "state": "Alabama", "latitude": 32.806671, "longitude": -86.79113, "no_travel_pop": 863024, "travel_pop": 4009910}, {"year": "2019", "month": "12", "month_abbr": "Dec", "month_year": "Dec-2019", "state": "Alabama", "latitude": 32.806671, "longitude": -86.79113, "no_travel_pop": 885949, "travel_pop": 3986985}, {"year": "2019", "month": "2", "month_abbr": "Feb", "month_year": "Feb-2019", "state": "Alabama", "latitude": 32.806671, "longitude": -86.79113, "no_travel_pop": 786528, "travel_pop": 4086406}, {"year": "2019", "month":